# 2 Solving the RFA problem using FEM

Before starting with the FEniCS code, we will present a short theoretical introduction. The simulation of RFA in atrial wall involves two main physical coupled problems modeled by the electric equation and the bioheat equation, we briefly introduce both. The thermal damage is evaluated by considering the Arrhenius equation which is commented aswell. The structure of this section of the tutorial is as follows: first the theory, then the strategy of the numerical solution, and finally, the code.

## 2.1 Theory

### 2.1.1 The electric equation

In RFA the heat ($q_{RF}$) is generated by Joule effect, applying an electric current to the desired region. This heat depends on the electric conductivity ($\sigma$) and the electric field ($E$), and is given by the equation $q_{\text{RF}}=\sigma \left| E\right|^{2}$. The electric field is defined as $E=-\nabla V$, where $V$ is the voltage. The latter is computed by solving the called Poisson equation in the desired volume (domain) $\Omega$:

$$\begin{equation} \nabla \cdot \left(\sigma \nabla V \right) = 0.\end{equation}\tag{1}$$

The electric conductivity can be a function of the position and the temperature. The Eq. (1) is a __*partial differential equation*__ (PDE) and can be solved if the __*boundary conditions*__ are known, in our case (remember the section of mesh generation) we have set:

- $V = 50$ V in the tagged with 2 (*Physical Line(2) = {1,2,4}; // active electrode (metal)*).
- $V = 0$ V in the tagged with 3 (*Physical Line(3) = {11,12}; // passive electrode, V = 0*).

These are Dirichlet boundary conditions. The domains of the boundary conditions will be called $\Gamma$, in particular for a Dirichlet boundary condition $\Gamma_{\text{D}}$.

### 2.1.2 The bioheat equation

The temperature $T$ is obtained by solving the governing PDE for the thermal problem, which is the bioheat equation modified by the enthalpy method to take the vaporization phenomenon into account:

$$\begin{equation} \frac{\partial \rho h}{\partial t} = \nabla \cdot \left( k \nabla T\right) +q_{\text{RF}} +\beta \rho_{\text{b}}c_{\text{b}}\omega_{\text{b}}\left(T_{\text{b}}-T\right)+Q_{\text{met}}\end{equation}\tag{2}$$

where $k$ is the thermal conductivity, $\rho$ is the density, $c$ is the specific heat, $\omega$ is the perfusion, and we set the $Q_{\text{met}}=0$ (metabolic heat) because it is negligible in RFA. The parameter $\beta$ is a coefficient which took the values of 0 and 1, according to the value of the thermal damage (we will explain it latter). The mentioned parameters can be functions of the position and temperature. The subscript b indicates that they are parameters of blood. Regarding the enthalpy term, the dependence with temperature is as follows:

$$\begin{equation}\frac{\partial \rho h}{\partial t} = \frac{\partial T}{\partial t} \cdot \begin{array}{l} \rho_{\text{l}} c_{\text{l}} \; \text{ if } \; T < 99 ^{\text{ o}}\text{C }\\h_{\text{fg}}C \; \text{  if } \; 99 \leq T \leq 100  ^{\text{ o}}\text{C }\\ \rho_{\text{g}}c_{\text{g}} \; \text{ if } \; T > 100  ^{\text{ o}}\text{C }\end{array} \end{equation} \tag{3}$$

The subscript l, and g refer to liquid state, and to gaseous state, respectively. $h_{\text{fg}}$ is the product of water latent heat of vaporization and water density at 100 $^{\text{ o}}\text{C}$, and $C$ is water content inside the tissue (68%).

The __*boundary conditions*__ of Eq. (2) are:

- $T = 37 ^{\text{ o}}\text{C}$ in the tagged with 3 and 4 (*Physical Line(3) = {11,12}; // passive electrode, U = 0 V and 37 deg* and *Physical Line(4) = {8,17,18}; // I = 0 A and 37 deg*).
- $T = 40 ^{\text{ o}}\text{C}$ in the tagged with 5 (*Physical Line(5) = {3}; // perfusion constant temperature 40 deg*). This simulates a irrigated electrode, perfused with saline solution at $40 ^{\text{ o}}\text{C}$.

These conditions are also Dirichlet. The thermal effect of the circulating blood in the cardiac chamber (assuming high flow) can be modeled using thermal transfer coefficients between blood and the tissue ($h_{\text{t}} = 610 \; \text{W}/\text{m}^{2} \text{K}$) and between blood and the electrode ($h_{\text{e}} = 3346 \; \text{W}/\text{m}^{2} \text{K}$). They are known as Robin boundary condition and are expressed as: $-\frac{\partial T}{\partial n}=h_{\text{i}}\left( T - T_{\text{b}}\right)$, for $\text{i}=\text{t},\text{e}$ where $n$ is the unit vector which points the normal direction of the boundary (line in 2D or surface in 3D). We set such a condition to the lines tagged with 6 and 7, respectively (*Physical Line(6) = {16}; // convective heat loss (plastic)
Physical Line(7) = {7}; // convective heat loss (tissue)*).

In the case of Robin and Neumman boundary conditions we will call them $\Gamma_{\text{R}}$ and $\Gamma_{\text{N}}$, respectively.

This problem is a transient heat equation, therefore __*initial condition*__ is also needed:
$$T\left(x,y, t = 0 \; \text{s}\right) = 37 ^{\text{ o}}\text{C},$$
we make implicit here the dependence on position and time.

### 2.1.3 The thermal damage

The Arrhenius damage model associates temperature with exposure time using a first‐order kinetics relationship:

$$\begin{equation} D\left(t\right) = \int_{0}^{t} Ae^{-\frac{\Delta E}{RT}} d\tau \end{equation} \tag{4},$$

where $A$ and $\Delta E$ are a frequency factor and the activation
energy for the irreversible damage reaction, respectively. $R$ is the universal gas constant. We assessed the thermal damage by the D99 isoline, which corresponds to $D= 4.6$ (99% probability of cell death). This means that when the value of $D=4.6$ we change the value of $\beta$ to zero, then the blood perfusion term of the bioheat equation is nulled. The location of the D99 isoline defined the size of the coagulation zone.
For the purpose of this tutorial we express the Eq. (4) as an ordinary differential equation (ODE):
$$\frac{dD}{dt}=Ae^{-\frac{\Delta E}{RT\left(t\right)}}\tag{5}.$$
Note that we explicit the dependence on time of the temperature ($T\left(t\right)$), in order to show that it results in a non-linear ODE that should be numerically solved at each time step.

### 2.1.4 The variational form

The starting point for the finite element methods is to express the PDE in *variational form* in order to solve the weak problem. For example, if we work with Eq. (1) (a little bit modified to generalize the reasoning, that is, $-\nabla \cdot \left(\sigma \nabla V \right) = f$, where $f$ is a known source), first we have to multiply the PDE by a function $v$ (__*test function*__) and integrate the resulting equation over the domain $\Omega$. 
$$-\nabla \cdot \left(\sigma \nabla V \right) = f \Rightarrow -\int_{\Omega}\nabla \cdot \left(\sigma \nabla V \right)\; v\; \text{dx} = \int_{\Omega} f \; v \; \text{dx}.$$
We denote here $\text{dx}$ as the differential of the domain $\Omega$, in this example: $\text{dx}=dxdy$. Note that the left part of Eq. (1) has second-order derivatives, solving PDEs with FEM requires integration by parts in order to obtain first-order derivatives only. This is obtained by applying the Green’s Formula.
$$-\int_{\Omega}\nabla \cdot \left(\sigma \nabla V \right)\; v\; \text{dx} = \int_{\Omega} \sigma\nabla V \cdot \nabla v \;\text{dx} - \int_{\Gamma} \sigma \; n\cdot \nabla V \;v \;\text{ds}.\tag{6}$$
The term $n\cdot\nabla V$ is the derivative of $V$ in the outward normal direction $n$ on the  respective boundary, and it is denoted as $\frac{\partial V}{\partial n}$. The differential $\text{ds}$ is defined on the boundary domain $\Gamma$. Reordering the Eq. (6):
$$\int_{\Omega} \sigma \nabla  V \cdot \nabla v \;\text{dx} = \int_{\Omega} f \; v \; \text{dx}-\int_{\Gamma} \sigma \; \frac{\partial V}{\partial n} \;v \;\text{ds}.\tag{7}$$
In our electrical problem there are only Dirichlet boundary conditions and $v$ can be chosen in order to vanish on the boundaries, therefore the second term of the right hand of Eq. (7) also vanishes: 
$$\int_{\Omega} \sigma \nabla  V \cdot \nabla v \;\text{dx} = \int_{\Omega} f \; v \; \text{dx}.\tag{8}$$
The left hand of this equation is called the __*bilinear form*__ and the right hand the __*linear form*__, and usually are denoted by: $a\left(V,v\right)$ and $L\left(v\right)$, respectively. Note that $a$ is function of both $V$ and $v$, and $L$ is only function of $v$ (because $f$ is the known source).

In the case of the bioheat equation the procedure is similar, with the addition of the Robin boundary conditions (convective). They are modeled as $-k\frac{\partial T}{\partial n}=h\left( T - T_{\text{b}}\right)$, such a condition is then written as:
$$-\int_{\Gamma_{\text{R}}} k \; \frac{\partial T}{\partial n} \;v \;\text{ds} = \int_{\Gamma_{\text{R}}} \; h\left( T - T_{\text{b}}\right)  \;v \;\text{ds}\tag{9}$$

The Eqs. (7-9) are written in FEniCS in a very similar way, for example Eq. (8) is coded as:

In [ ]:
a = sigma*inner(nabla_grad(V), nabla_grad(v))*dx
L = f*v*dx

and to solve it the code is:

In [ ]:
solve(a == L, V, bcs)

This code is read as: make *a* equal to *L* with the boundary conditions *bcs* and save the result in variable *V*.



## 2.2 Numerical strategy

It should be remarked that we have to solve three coupled equations: two PDEs and one ODE. We need to discretize them in space and time. In this section we comment the time discretization.

### 2.2.1 General approach

Note that Eq. (1) is an electric quasi‐static approach, which is used since the resistive current is much lower than the displacement current at RF. Equation (1) is only related to the Eq. (2) by the dependence on $T$ of the electric conductivity ($\sigma \left(T\right)$). Therefore, one approach is to solve Eq. (1) with a known temperature by considering the electric conductivity constant at this particular temperature. That is, picewise constant within the time step, starting from the body temperature ($37 ^{\text{ o}}\text{C}$). In this way the voltage is uncoupled from the temperature, it is considered a weak coupling. Therefore the general approach can be outlined by the next __*steps*__:
1. Initialize temperature of the geometry.
2. With known temperatures solve Eq. (1) to obtain $q_{\text{RF}}$.
3. With known $q_{\text{RF}}$ solve Eq. (2).
4. With known temperature integrate the Eq. (4).
5. Go to step 2 and repeat.

### 2.2.2 Scheme of the time discretisation

Equation (2) depends on time, it can be solved by a simple backward Euler approach. The first derivative is approached by:
$$\begin{equation}\left(\frac{\partial T}{\partial t}\right)^{\left(n+1\right)}\approx \frac{T^{\left(n+1\right)}-T^{\left(n\right)}}{\Delta t} \end{equation},\tag{8}$$
where the superscript $\left(n\right)$ indicates the time step $n$. The first step is known, because we have the initial condition $T\left(x,y, t = 0 \; \text{s}\right) = 37 ^{\text{ o}}\text{C}$.

### 2.2.3 Dependence of temperature of the parameters

For clarity in the example of this tutorial, we consider *two parameters* which depend on temperature: the electric conductivity and the enthalpy. The latter is modeled with the Eq. (3), and we model the conductivity with a linear dependence on temperature. That is: $\sigma\left( T\right)= \sigma\left(37 ^{\text{ o}}\text{C}\right)\cdot \left( 1+0.02 \frac{1}{^{\text{ o}}\text{C}} \cdot T\right)$.

## 2.3 Coding

The first line in the program should be:

In [ ]:
from fenics import *

with this sentence we import all the FEniCS functions. As the mesh has already been created in the previous tutorial, we can create the finite element __function space__ $\Omega$ where all the variables will "live". In the program  we will call it *W*.

In [ ]:
W = FunctionSpace(mesh, 'CG', 2)

Here, "CG" stands for Continuous Galerkin, implying the standard Lagrange family of elements, in this case with degree 2 (go to this [link](http://femtable.org/) to see the notation).

Then, to initialize the temperature in the whole mesh we write:

In [ ]:
T_initial = Constant(310.15)
T = project_axisym(T_initial,W)

Note that we express the temperature in Kelvin and we use the function *Constant* which indicates to FEniCS that this is a constant. We create a function *project_axisym* to solve the problem in a axisymmetric way, with main axis the vertical center of the electrode. As a result the reader will see that most of the terms will be multiplied by *r* (distance to the axis of symmetry), which is defined as:

In [ ]:
r = Expression('x[0]',degree=2)#Define the radius for axisymetric problem

In the section 2.2.1 we have sketched the algorithm, it was mentioned that the electric problem and the bioheat equation are weakly coupled then we define two main functions: *electric_problem* and *rfa_bioheat_problem*.

### 2.3.1 Function *electric_problem*

We describe this function in order to show the general form of a FEniCS code. First, we define the parameters *Vprobe* and *source* which are the value of the active electrode voltage and the value of $f$ (in our example is zero). Then, we load the mesh, domains and boundaries, and define of boundary conditions:

In [ ]:
Vprobe = Constant(55.0) #active electrode voltage
source = Constant(0.0) #the source of the electric problem is set to zero
mesh = Mesh(filename+".xml")
boundaries = MeshFunction("size_t", mesh, filename+"_facet_region.xml")
subdomains = MeshFunction("size_t", mesh, filename+"_physical_region.xml")
bcs = []
bcs.append(DirichletBC(W, Constant(0.0), boundaries, 3))#passive electrode
bcs.append(DirichletBC(W, Vprobe, boundaries, 2))#active electrode
bcs.append(DirichletBC(W, Vprobe, boundaries, 5))#active electrode

Note that it is defined as a python list where the Dirichlet boundary conditions are appended. Only Dirichlet type can be defined in this step, later we will show how define Robin and Neumman type conditions.

The defintion of the electric conductivity map is as follows:

In [ ]:
class SIGMA(UserExpression):
    def __init__(self, subdomains,temperature,sigma_1, sigma_2,sigma_3, sigma_4, **kwargs):
        super().__init__(**kwargs)
        self.subdomains = subdomains
        self.temperature = temperature
        self.sigma_1 = sigma_1 # atrial wall
        self.sigma_2 = sigma_2 # connective tissue
        self.sigma_3 = sigma_3 # plastic
        self.sigma_4 = sigma_4 # blood   
    def eval_cell(self, values, x, cell):
        if self.subdomains[cell.index] == 1:
            values[0] = (self.sigma_1)*(1.+0.02*self.temperature(x[0],x[1]))# atrial wall
        elif self.subdomains[cell.index] == 2:
            values[0] = (self.sigma_2)*(1.+0.02*self.temperature(x[0],x[1]))# connective tissue
        elif self.subdomains[cell.index] == 3:
            values[0] = self.sigma_3# plastic
        else:
            values[0] = self.sigma_4# blood

Note that it is a class derived from the *UserExpression* class and the inputs are: the *subdomains* (with the tags that mark each region), the *temperature* map (which is updated each time step), and the parameters at $37 ^{\text{ o}}\text{C}$ within each subdomain *sigma_1*, *sigma_2*, *sigma_3*, and *sigma_4* (we already have defined them as Constants). We also remark that the electric conductivities that depend on the temperature are the atrial wall and the connective tissue.
Then we call this class as:

In [ ]:
sigma = SIGMA(subdomains,T,sigma_1, sigma_2,sigma_3, sigma_4, degree=0)

The degree = 0 indicates that it is a scalar function. To check if values were set in the right way:

In [ ]:
File("outputs/sigma.pvd") << project_axisym(sigma,W)

The previous sentence creates the fold *outputs* and saves the function of space (sigma) in a file called sigma.pvd which can be visualized by Paraview.

![Conductivity map](figure1.png "Conductivity map")

Then we define two functions that "live" in the space $\Omega$ (called W in the program):

In [ ]:
V = TrialFunction(W)
v = TestFunction(W)

These are the trial function and the test function ($v$). The trial function is taken as a symbolic variable (is the unknown function). Therefore we can define now the variational form:

In [ ]:
a = sigma*inner(nabla_grad(V), nabla_grad(v))*r*dx
L = source*v*r*dx

We remark that the formulas in the variational formulation translate directly to very similar Python code. The language used to express weak forms is called UFL (Unified Form Language) and is an integral part of FEniCS. Now, to compute the voltage we first have to convert the *trial function V* into *function* in order to indicates to FEniCS that it should be numerically solved.

In [ ]:
V = Function(W)
solve(a == L, V, bcs)
File("outputs/voltage.pvd") << V

Plotted with Paraview it gives:

![Voltage](figure2.png "Voltage")

As we are interested in computing the heat generated by RF, then the quantity needed is the $q_{\text{RF}}$. We make this computation using FEM aswell, by $q_{\text{RF}}=\sigma \nabla V \cdot \nabla V$.

In [ ]:
v = TestFunction(W)
qRF = TrialFunction(W)
a = qRF*v*r*dx
L = v*sigma*inner(nabla_grad(V), nabla_grad(V))*r*dx
qRF = Function(W)
solve(a == L, qRF)

The $q_{\text{RF}}$ will be used as input for the function *rfa_bioheat_problem*.

### 2.3.2 Function *rfa_bioheat_problem*

